In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ MorphoMol
  1 dependency successfully precompiled in 3 seconds. 243 already precompiled.
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]
Precompiling GLMakie
  ✓ MeshIO
  ✓ Distributions → DistributionsChainRulesCoreExt
  ✓ KernelDensity
  ✓ Makie
  ✓ GLMakie
  5 dependencies successfully precompiled in 60 seconds. 236 already precompiled.
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
┌ Warning: Module Distributions with build ID fafbfcfd-1ae4-ac42-0000-13a9f4fb4aae is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a].
Precompiling Makie
  ✓ GridLayoutBase
  ✓ Makie
  2 dependencies successfully 

In [2]:
function get_tps(dgms)
    [MorphoMol.Energies.get_total_persistence(dgm) for dgm in dgms]
end
    
function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [3]:
function get_configurations_from_folder(folder)
    files = readdir(folder)
    realizations = [Vector{Vector{Vector{Float64}}}([]) for i in 1:length(files)]
    for file in readdir(folder)
        centers = txt_to_points("$(folder)$(file)")
        n_atoms_per_mol = Int(length(centers) / 2)
        c1 = centers[1:n_atoms_per_mol]
        c2 = centers[n_atoms_per_mol+1:end]
        realizations[parse(Int, split(file, ".")[1]) + 1] = [c1, c2]
    end
    realizations
end

get_configurations_from_folder (generic function with 1 method)

In [4]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    py_dgms = MorphoMol.Energies.get_interface_diagrams(folder)
    dgms = [[dgm[1], dgm[2], dgm[3], dgm[4]] for dgm in py_dgms]
    tps = get_tps(dgms)
    Es = get_fsols(folder)
    thetas = get_thetas(folder)
    realizations = get_configurations_from_folder(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps Es thetas realizations
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [75]:
# folder = "../../Data/measure_paths/txts/6r7m/"
# for subdir in readdir(folder)
#     if subdir != ".DS_Store"
#         folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
#     end
# end

# folder = "../../Data/measure_paths/txts/6r7m/1/"
# dgms = get_interface_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# dgms_selection = [[dgm[2]; dgm[3]] for dgm in dgms]
# tps = get_tps(dgms_selection)
# Es = get_fsols(folder)
# thetas = get_thetas(folder);

In [10]:
@load "../../Data/measure_paths/jld2s/6r7m/19.jld2" dgms tps Es thetas realizations
#tps = [get_total_persistence(dim_dgms, -0.1) for dim_dgms in dgms] #Recalculating due to weights!
tps = get_tps(dgms)
xs = [i for i in 1:length(dgms)];

In [11]:
@load "../../Data/measure_paths/jld2s/6r7m/19.jld2" dgms tps Es thetas realizations
#tps = [get_total_persistence(dim_dgms, -0.1) for dim_dgms in dgms] #Recalculating due to weights!
tps = get_tps(dgms)
xs = [i for i in 1:length(dgms)];
two_ps = [MorphoMol.Energies.get_persistence(dgm[2], -0.1) for dgm in dgms]
three_ps = [MorphoMol.Energies.get_persistence(dgm[3], -0.1) for dgm in dgms]
tps = [MorphoMol.Energies.get_total_persistence(dgm, -0.1) for dgm in dgms];

In [12]:
# SLOWS DOWN VISUALIZATION SUBSTANTIALLY WHEN INCLUDED.

# function get_density_from_dgm(xs,ys,dgm,search_radius = 5.0)
#     tree = KDTree(transpose(dgm))
#     [length(inrange(tree, [x, y], search_radius)) for x in xs, y in ys]
# end

# #Persistence Heatmaps 
# pd_hm_ax_a = Axis(f[2, 1], title = "Persistence codim = 2")
# pd_hm_ax_b = Axis(f[2, 2], title = "Persistence codim = 3")

# sr = 0.5
# hs = [e for e in 0.0:0.25:25.0]
# vs = [e for e in 0.0:0.25:25.0]
# hm_a = @lift([get_density_from_dgm(hs, vs, dim_dgms[2], sr) for dim_dgms in dgms][$x])
# hm_b = @lift([get_density_from_dgm(hs, vs, dim_dgms[3], sr) for dim_dgms in dgms][$x])

# contourf!(pd_hm_ax_a, hs, vs, hm_a, levels = 10)
# contourf!(pd_hm_ax_b, hs, vs, hm_b, levels = 10)

get_density_from_dgm (generic function with 2 methods)

In [17]:
f = Figure(fontsize = 7)

#Slider
sl_i = Slider(f[4, 1:4], range = 1:length(dgms), startvalue = 1)
x = sl_i.value

# Persistence Diagrams
pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2", xticks = 0:25:100, yticks = 0:25:100)
pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3", xticks = 0:25:100, yticks = 0:25:100)
dgm_points_2 = @lift([Point2f(dgms[$x][2][i,1], dgms[$x][2][i,2]) for i in 1:size(dgms[$x][2])[1]])
dgm_points_3 = @lift([Point2f(dgms[$x][3][i,1], dgms[$x][3][i,2]) for i in 1:size(dgms[$x][3])[1]])

ms = 5
scatter!(pd_ax_a, dgm_points_2, color = :orange, markersize = ms)
lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
scatter!(pd_ax_b, dgm_points_3, color = :green, markersize = ms)
lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

#Energy and Theta
theta_ax = Axis(f[2, 1], title = L"\Theta")
E_ax = Axis(f[2, 2], title = L"F_{sol}")

theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
E_mark = @lift(Point2f($x, $@lift(Es[$x])))

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

scatter!(E_ax, xs, Es, color=:blue, markersize = ms)
scatter!(E_ax, E_mark, color=:black, markersize = 10)


# Configuration
conf_ax = Axis3(f[1:2, 3:4])
c1_points = @lift([Point3f(realizations[$x][1][i][1], realizations[$x][1][i][2], realizations[$x][1][i][3]) for i in 1:length(realizations[$x][1])])
c2_points = @lift([Point3f(realizations[$x][2][i][1], realizations[$x][2][i][2], realizations[$x][2][i][3]) for i in 1:length(realizations[$x][2])])

conf_ms = 10
scatter!(conf_ax, c1_points, markersize = conf_ms)
scatter!(conf_ax, c2_points, markersize = conf_ms)

#Persistence Measures
two_p_ax = Axis(f[3, 1], title = "codim 2 persistence")
three_p_ax = Axis(f[3, 2], title = "codim 3 persistence")
tp_ax = Axis(f[3, 3], title = "Total Persistence")

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))
three_p_mark = @lift(Point2f($x, $@lift(three_ps[$x])))

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(two_p_ax, xs, two_ps, color=:orange, markersize = ms)
scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

scatter!(three_p_ax, xs, three_ps, color=:green, markersize = ms)
scatter!(three_p_ax, three_p_mark, color=:black, markersize = 10)

display(f)


GLMakie.Screen(...)